In [3]:
import os
import numpy as np
import pandas as pd
import pickle

In [13]:
def save_pickle(data, file_name):
    """
    Saves data as pickle format
    """
    with open(file_name, 'wb') as f:
        pickle.dump(data, f)
    return None

In [14]:
cwd = os.getcwd()
parent_wd = cwd.replace('/preprocessing', '')
hold_feature_path = parent_wd + '/raw_data/HoldFeature2016.csv'
training_set_path = parent_wd + '/preprocessing/training_set'
dev_set_path = parent_wd + '/preprocessing/dev_set'
test_set_path = parent_wd + '/preprocessing/test_set'

In [15]:
with open(training_set_path, 'rb') as f:
    training_set = pickle.load(f)
with open(dev_set_path, 'rb') as f:
    dev_set = pickle.load(f)
with open(test_set_path, 'rb') as f:
    test_set = pickle.load(f)

In [4]:
features = pd.read_csv(hold_feature_path, dtype=str)

# convert features from pd dataframe to dictionary
feature_dict = {}
for index in features.index:
    feature_item = features.loc[index]
    feature_dict[(int(feature_item['X_coord']), int(feature_item['Y_coord']))] = np.array(
        list(feature_item['Difficulties'])).astype(int)

In [6]:
feature_array = np.zeros((len(feature_dict), 8))
i = 0
for key, values in feature_dict.items():
    feature_array[i, 0:6] = values
    feature_array[i, 6:] = key
    i += 1

In [10]:
mu = feature_array.mean(axis = 0)
std = np.std(feature_array, axis = 0)

In [36]:
def normalization(input_set):
    mask = np.zeros_like(input_set['X'])
    for i in range(len(mask)):
        mask[i, 0:int(input_set['tmax'][i]), :] = 1
    
    X_normalized = np.copy(input_set['X'])
    X_normalized[:, :, 0:8] -= mu
    X_normalized[:, :, 0:8] /= std
    X_normalized *= mask
    
    output_set = input_set
    output_set['X'] = X_normalized
    return output_set

In [37]:
training_set_normalized = normalization(training_set)
dev_set_normalized = normalization(dev_set)
test_set_normalized = normalization(test_set)

In [41]:
training_normalized_set_path = parent_wd + '/preprocessing/training_set_n'
dev_normalized_set_path = parent_wd + '/preprocessing/dev_set_n'
test_normalized_set_path = parent_wd + '/preprocessing/test_set_n'
save_pickle(training_set_normalized, training_normalized_set_path)
save_pickle(dev_set_normalized, dev_normalized_set_path)
save_pickle(test_set_normalized, test_normalized_set_path)